In [ ]:
! git clone https://github.com/iwyoo/LSTM-autoencoder

In [1]:
import tensorflow as tf
from tensorflow.python.ops.rnn_cell import LSTMCell

import numpy as np


class LSTMAutoencoder(object):

    """Basic version of LSTM-autoencoder.
  (cf. http://arxiv.org/abs/1502.04681)
  Usage:
    ae = LSTMAutoencoder(hidden_num, inputs)
    sess.run(ae.train)
  """

    def __init__(
        self,
        hidden_num,
        inputs,
        cell=None,
        optimizer=None,
        reverse=True,
        decode_without_input=False,
        ):
        """
    Args:
      hidden_num : number of hidden elements of each LSTM unit.
      inputs : a list of input tensors with size 
              (batch_num x elem_num)
      cell : an rnn cell object (the default option 
            is `tf.python.ops.rnn_cell.LSTMCell`)
      optimizer : optimizer for rnn (the default option is
              `tf.train.AdamOptimizer`)
      reverse : Option to decode in reverse order.
      decode_without_input : Option to decode without input.
    """

        self.batch_num = inputs[0].get_shape().as_list()[0]
        self.elem_num = inputs[0].get_shape().as_list()[1]

        if cell is None:
            self._enc_cell = LSTMCell(hidden_num)
            self._dec_cell = LSTMCell(hidden_num)
        else:
            self._enc_cell = cell
            self._dec_cell = cell

        with tf.variable_scope('encoder'):
            (self.z_codes, self.enc_state) = tf.contrib.rnn.static_rnn(self._enc_cell, inputs, dtype=tf.float32)

        with tf.variable_scope('decoder') as vs:
            dec_weight_ = tf.Variable(tf.truncated_normal([hidden_num,
                    self.elem_num], dtype=tf.float32), name='dec_weight'
                    )
            dec_bias_ = tf.Variable(tf.constant(0.1,
                                    shape=[self.elem_num],
                                    dtype=tf.float32), name='dec_bias')

            if decode_without_input:
                dec_inputs = [tf.zeros(tf.shape(inputs[0]),
                              dtype=tf.float32) for _ in
                              range(len(inputs))]
                (dec_outputs, dec_state) = tf.contrib.rnn.static_rnn(self._dec_cell, dec_inputs, initial_state=self.enc_state,
                        dtype=tf.float32)
                if reverse:
                    dec_outputs = dec_outputs[::-1]
                dec_output_ = tf.transpose(tf.stack(dec_outputs), [1, 0,
                        2])
                dec_weight_ = tf.tile(tf.expand_dims(dec_weight_, 0),
                        [self.batch_num, 1, 1])
                self.output_ = tf.matmul(dec_output_, dec_weight_) + dec_bias_
            else:

                dec_state = self.enc_state
                dec_input_ = tf.zeros(tf.shape(inputs[0]),
                        dtype=tf.float32)
                dec_outputs = []
                for step in range(len(inputs)):
                    if step > 0:
                        vs.reuse_variables()
                    (dec_input_, dec_state) = \
                        self._dec_cell(dec_input_, dec_state)
                    dec_input_ = tf.matmul(dec_input_, dec_weight_) \
                        + dec_bias_
                    dec_outputs.append(dec_input_)
                if reverse:
                    dec_outputs = dec_outputs[::-1]
                self.output_ = tf.transpose(tf.stack(dec_outputs), [1,
                        0, 2])

        self.input_ = tf.transpose(tf.stack(inputs), [1, 0, 2])
        self.loss = tf.reduce_mean(tf.square(self.input_
                                   - self.output_))

        if optimizer is None:
            self.train = tf.train.AdamOptimizer().minimize(self.loss)
        else:
            self.train = optimizer.minimize(self.loss)


In [2]:
# Basic libraries
import numpy as np
import tensorflow as tf

tf.reset_default_graph()
tf.set_random_seed(2016)
np.random.seed(2016)


In [4]:

# Constants
batch_num = 128
hidden_num = 12
step_num = 67
elem_num = 2000
iteration = 10000

In [9]:
import pandas as pd
notifs_df = pd.read_csv('/Users/vahid/data/recommender/notifs_corrected.csv')

input_data = np.zeros([notifs_df.shape[0], 2000])

for index, row in notifs_df.iterrows():
    if type(row.text) == str:
        words = [int(word) for word in row.text.split(' ')]
        for idx, word in enumerate(words):
            input_data[index, word] = 1 
            
input_data.shape
    

(6347, 2000)

In [6]:

# placeholder list
p_input = tf.placeholder(tf.float32, shape=(batch_num, step_num, elem_num))
p_inputs = [tf.squeeze(t, [1]) for t in tf.split(p_input, step_num, 1)]


In [7]:
cell = tf.nn.rnn_cell.LSTMCell(hidden_num, use_peepholes=True)
ae = LSTMAutoencoder(hidden_num, p_inputs, cell=cell, decode_without_input=True)


In [8]:
import time
from tqdm import tqdm
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for i in range(iteration):
        """Random sequences.
          Every sequence has size batch_num * step_num * elem_num 
          Each step number increases 1 by 1.
          An initial number of each sequence is in the range from 0 to 19.
          (ex. [8. 9. 10. 11. 12. 13. 14. 15])
        """
#         r = np.random.randint(20, size=batch_num).reshape([batch_num, 1, 1])
#         r = np.tile(r, (1, step_num, elem_num))
        
#         d = np.linspace(0, step_num, step_num, endpoint=False).reshape([1, step_num, elem_num])
#         d = np.tile(d, (batch_num, 1, 1))
#         random_sequences = r + d
#         print(random_sequences.shape)
        s_time = time.time()
        for batch_idx in tqdm(range(input_data.shape[0] // batch_num)):
            batch_data = input_data[batch_idx * batch_num : ( batch_idx + 1 ) * batch_num]
            (loss_val, _) = sess.run([ae.loss, ae.train], {p_input: batch_data})
        print('iter %d:' % (i + 1), loss_val, time.time() - s_time)

    (input_, output_) = sess.run([ae.input_, ae.output_], {p_input: r + d})
    print('train result :')
    print('input :', input_[0, :, :].flatten())
    print('output :', output_[0, :, :].flatten())


  0%|          | 0/49 [00:00<?, ?it/s]

iter 1: 0.0030840177 45.44552302360535


  0%|          | 0/49 [00:00<?, ?it/s]

iter 2: 0.0006657991 40.75128197669983


  0%|          | 0/49 [00:00<?, ?it/s]

iter 3: 0.00019930373 37.22128891944885


 31%|███       | 15/49 [00:12<00:27,  1.22it/s]

KeyboardInterrupt: 

In [52]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from keras.layers import Lambda, Input, Dense
from keras.models import Model
from keras.datasets import mnist
from keras.losses import mse, binary_crossentropy
from keras.utils import plot_model
from keras import backend as K

import numpy as np
import matplotlib.pyplot as plt
import argparse
import os

from sklearn.model_selection import train_test_split

# this is the size of our encoded representations
encoding_dim = 8   # 32 floats -> compression factor 24.5, assuming the input is 784 floats

# this is our input placeholder; 784 = 28 x 28
input_dim = 2000

my_epochs = 100

encoded = Dense(512, activation='relu')(x_train)
encoded = Dense(64, activation='relu')(encoded)
encoded = Dense(latent_dim, activation='relu')(encoded)

# "decoded" is the lossy reconstruction of the input
decoded = Dense(64, activation='relu')(encoded)
decoded = Dense(512, activation='relu')(decoded)
decoded = Dense(x_train.shape[1], activation='sigmoid')(decoded)

# this model maps an input to its reconstruction
autoencoder = Model(input_img, decoded)

# Separate Encoder model

# this model maps an input to its encoded representation
encoder = Model(input_img, encoded)

# Separate Decoder model

# create a placeholder for an encoded (32-dimensional) input
encoded_input = Input(shape=(encoding_dim, ))
# retrieve the layers of the autoencoder model
decoder_layer1 = autoencoder.layers[-3]
decoder_layer2 = autoencoder.layers[-2]
decoder_layer3 = autoencoder.layers[-1]
# create the decoder model
decoder = Model(encoded_input, decoder_layer3(decoder_layer2(decoder_layer1(encoded_input))))

# Train to reconstruct MNIST digits

# configure model to use a per-pixel binary crossentropy loss, and the Adadelta optimizer
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

(5500, 2000) (847, 2000)


ValueError: Layer dense_39 was called with an input that isn't a symbolic tensor. Received type: <class 'numpy.ndarray'>. Full input: [array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]], dtype=float32)]. All inputs to the layer should be tensors.

In [19]:
# VAE loss = mse_loss or xent_loss + kl_loss

reconstruction_loss = mse(inputs, outputs)

reconstruction_loss *= original_dim
vae_loss = K.mean(reconstruction_loss)
vae.add_loss(vae_loss)
vae.compile(optimizer='adam')
vae.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_input (InputLayer)   (None, 2000)              0         
_________________________________________________________________
encoder (Model)              [(None, 2), (None, 2), (N 1026564   
_________________________________________________________________
decoder (Model)              (None, 2000)              1027536   
Total params: 2,054,100
Trainable params: 2,054,100
Non-trainable params: 0
_________________________________________________________________


In [20]:
vae.fit(x_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(x_test, None))


Train on 5500 samples, validate on 847 samples
Epoch 1/50
5500/5500 [==============================] - 4s 749us/step - loss: 187.6390 - val_loss: 18.8190
Epoch 2/50
5500/5500 [==============================] - 2s 387us/step - loss: 16.5498 - val_loss: 16.4903
Epoch 3/50
5500/5500 [==============================] - 3s 480us/step - loss: 15.7478 - val_loss: 15.7258
Epoch 4/50
5500/5500 [==============================] - 3s 457us/step - loss: 15.4137 - val_loss: 15.4451
Epoch 5/50
5500/5500 [==============================] - 3s 480us/step - loss: 15.5207 - val_loss: 15.2056
Epoch 6/50
5500/5500 [==============================] - 2s 387us/step - loss: 15.3719 - val_loss: 15.3043
Epoch 7/50
5500/5500 [==============================] - 2s 386us/step - loss: 15.2468 - val_loss: 15.0886
Epoch 8/50
5500/5500 [==============================] - 2s 403us/step - loss: 15.2565 - val_loss: 15.1823
Epoch 9/50
5500/5500 [==============================] - 2s 387us/step - loss: 15.1128 - val_loss: 15.125

In [43]:
prediction = vae.predict(input_data[:100])

In [44]:
for i in range(100):
    print(input_data[i].sum(), prediction[i].sum())

12.0 1.0
12.0 1.0
7.0 0.99992776
20.0 1.0
12.0 1.0
12.0 0.0
17.0 1.0
9.0 7.1602185e-06
17.0 1.0
17.0 0.0
12.0 1.0
25.0 1.0
17.0 0.0
25.0 1.0
25.0 1.0
12.0 1.0
11.0 0.0
17.0 1.0
20.0 1.0
12.0 1.0
12.0 1.0
12.0 1.0
17.0 1.0
4.0 0.09478525
12.0 1.0
15.0 1.0
15.0 1.0
5.0 3.0256373e-08
5.0 0.24617836
15.0 1.0
12.0 1.0
12.0 1.0
5.0 4.85787e-10
20.0 1.0
17.0 0.0
5.0 1.6034614e-29
10.0 0.0
17.0 0.0
14.0 0.0
12.0 1.0
12.0 1.0
12.0 1.0
9.0 1.2562422e-10
16.0 2.8970715e-10
12.0 1.0
18.0 0.0
18.0 1.0
10.0 0.0
12.0 1.0
6.0 0.0010068875
11.0 3.2111261e-31
17.0 1.0
10.0 0.0
12.0 1.0
9.0 2.0049497e-35
9.0 0.000599146
9.0 0.09645598
18.0 1.0
7.0 1.0
9.0 0.00010069027
13.0 1.0
24.0 0.0
9.0 9.037526e-30
9.0 3.5122136e-10
9.0 0.29444292
11.0 9.936364e-13
7.0 1.0
11.0 0.0
3.0 4.5814584e-08
11.0 0.83246714
9.0 6.70416e-09
11.0 0.0
13.0 1.0
12.0 1.0
11.0 3.811254e-32
11.0 1.6682986e-08
15.0 1.0
12.0 0.91614914
11.0 1.9804025e-09
7.0 1.0
16.0 1.0
10.0 0.999869
10.0 1.0
10.0 1.0
16.0 1.0
15.0 1.0
16.0 1.0
14.0

In [41]:
out = ''
for i in prediction[0]:
    out += str(int(i))
outinput

'010000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000